In [4]:
from quint.transcribtion.highlights import create_embedding, create_df
from quint.chunk.chunking import get_middle_points


path = '../data/podcast_transcript.txt'
# Split text to sentences and get their embedding, we use version=2 here because it specifies that the input is text
sentences, embeddings = create_embedding(path)
# Create a dataframe which returns sentences with generated timestamps
df = create_df(sentences, embeddings)
# Get the points where we need to split the text
true_middle_points = get_middle_points(embeddings)
# Initiate text to append to
text = ''
for num, each in enumerate(df['sentence']):
    # Chunk the text
    # If the index of the row is equal to a split point, add two new lines to the text
    if num in true_middle_points[0]:
        text += f' \n \n {each}'
    else:
        # Append a new line of text with no new line if it is not in the splitting points list
        text += f'{each}'
# Split text by new lines notation to get a list of texts - paragraphs
clean_chunks = text.split('\n \n')

/Users/poloniki/.pyenv/versions/3.10.6/envs/quint/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Embeddings for highlights created


In [14]:
import pandas as pd
import numpy as np
# Library to import pre-trained model for sentence embeddings
# Calculate similarities between sentences

# package for finding local minimas
from scipy.signal import argrelextrema
import jax.numpy as jnp


def rev_sigmoid_jax(x: jnp.ndarray) -> jnp.ndarray:
    return 1 / (1 + jnp.exp(0.5 * x))


def normalize_jax(X):
    l2_norm = jnp.linalg.norm(X, axis=1, keepdims=True)
    return X / l2_norm

def cosine_similarity_jax(X, dense_output=True):
    X_normalized = normalize_jax(X)
    K = jnp.dot(X_normalized, X_normalized.T)
    return K


p_size=10
similarities = cosine_similarity_jax(embeddings)
# To create weights for sigmoid function we first have to create space. P_size will determine number of sentences used and the size of weights vector.
x = jnp.linspace(-10,10,p_size)
# Then we need to apply activation function to the created space
y = jnp.vectorize(rev_sigmoid_jax)

# # Because we only apply activation to p_size number of sentences we have to add zeros to neglect the effect of every additional sentence and to match the length ofvector we will multiply
# activation_weights = jnp.pad(y(x),(0,similarities.shape[0]-p_size))

# ### 1. Take each diagonal to the right of the main diagonal
# diagonals = [similarities.diagonal(each) for each in range(0,similarities.shape[0])]
# ### 2. Pad each diagonal by zeros at the end. Because each diagonal is different length we should pad it with zeros at the end
# diagonals = [jnp.pad(each, (0,similarities.shape[0]-len(each))) for each in diagonals]
# ### 3. Stack those diagonals into new matrix
# diagonals = jnp.stack(diagonals)




# ### 4. Apply activation weights to each row. Multiply similarities with our activation.
# diagonals = diagonals * activation_weights.reshape(-1,1)
# ### 5. Calculate the weighted sum of activated similarities
# activated_similarities = jnp.sum(diagonals, axis=0)


# ### 6. Find relative minima of our vector. For all local minimas and save them to variable with argrelextrema function
# minmimas = argrelextrema(activated_similarities, jnp.less, order=2) #order parameter controls how frequent should be splits. I would not reccomend changing this parameter.




In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import WebBaseLoader
from langchain.chains.summarize import load_summarize_chain

loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
docs = loader.load()

llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-16k")
chain = load_summarize_chain(llm, chain_type="stuff")

chain.run(docs)

'The article discusses the concept of building autonomous agents powered by large language models (LLMs). It explores the components of such agents, including planning, memory, and tool use. The article provides case studies and proof-of-concept examples of LLM-powered agents in various domains. It also highlights the challenges and limitations of using LLMs in agent systems.'

In [10]:
path = '../data/podcast_transcript.txt'

with open(path) as f:
    doc = f.read()

In [14]:
from langchain.docstore.document import Document

docs = [Document(page_content=doc)]


In [16]:
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents.stuff import StuffDocumentsChain

# Define prompt
prompt_template = """Write a concise summary of the following:
"{text}"
CONCISE SUMMARY:"""
prompt = PromptTemplate.from_template(prompt_template)

# Define LLM chain
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-16k")
llm_chain = LLMChain(llm=llm, prompt=prompt)

# Define StuffDocumentsChain
stuff_chain = StuffDocumentsChain(
    llm_chain=llm_chain, document_variable_name="text"
)

# Single text
single_text = "Your single text here."

# Summarize the single text
summary = stuff_chain.run(docs)
print(summary)


InvalidRequestError: This model's maximum context length is 16385 tokens. However, your messages resulted in 22711 tokens. Please reduce the length of the messages.

In [18]:
question_prompt_template = """
                  Please provide a summary of the following text.
                  TEXT: {text}
                  SUMMARY:
                  """

question_prompt = PromptTemplate(
    template=question_prompt_template, input_variables=["text"]
)

refine_prompt_template = """
              Write a concise summary of the following text delimited by triple backquotes.
              Return your response in bullet points which covers the key points of the text.
              ```{text}```
              BULLET POINT SUMMARY:
              """

refine_prompt = PromptTemplate(
    template=refine_prompt_template, input_variables=["text"]
)

In [19]:
refine_chain = load_summarize_chain(
    docs,
    chain_type="refine",
    question_prompt=question_prompt,
    refine_prompt=refine_prompt,
    return_intermediate_steps=True,
)

ValidationError: 1 validation error for LLMChain
llm
  value is not a valid dict (type=type_error.dict)

In [22]:
map_reduce_outputs = map_reduce_chain({"input_documents": pages})


NameError: name 'map_reduce_chain' is not defined